# Parquet to Dask
load a parquet dataset into as dask cluster

Once the client connects to the dask cluster we can check for package consistency by calling `dask_client.get_versions(check=True)`.  Scheduler and worker
nodes are guaranteed to have the same package versions in `mlrun` since mismatches would likely cause serialization failures.  The client (in this notebook or in some other process) may have different versions and this mismatch is **less likely** to cause issues.  The following installs can be performed to ensure this mismatch becomes **unlikely** (as of this writing). **This may need to be done periodically as the dockerfiles refresh, unless you specifiy an image tag.**

    !python -m pip install --upgrade pip
    !python -m pip uninstall -y cmake
    !python -m pip install cmake
    !python -m pip install scikit-build

    !python -m pip install lz4
    !python -m pip install git+https://github.com/Blosc/python-blosc.git@v1.8.3
    !python -m pip install msgpack-python    
    !python -m pip install cloudpickle==1.2.2

    !python -m pip install numpy==1.17.4

## clean up
delete stale pods

In [1]:
# !mlrun clean -p -r

In [2]:
import mlrun

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters


In [3]:
DESCRIPTION        = 'load parquet dataset into a dask cluster'       

IMAGE              = 'yjbds/mlrun-dask:dev'                           # custom image
JOB_KIND           = 'dask'                                           

TASK_NAME          = 'user-task-parq-to-dask'                         # whatever

ARTIFACTS_PATH     = '/User/repos/demos/dask/artifacts'
YAML_PATH          = '/User/repos/demos/dask/yaml'

DATA_PATH          = '/User/repos/demos/dask/dataset/partitions'
DOWNSAMPLE         = 0.01
PARTITION_COLS     = ['Year', 'Month']
PARTITION_COLS     = None

DASK_SHARDS        = 8
DASK_THREADS_PER   = 8
MEMORY_LIMIT       = '5GB'

DASK_KEY           = 'airlines'

## load and configure function

In [4]:
FUNC_PY   = '/User/repos/demos/dask/code/parquet-to-dask.py'
FUNC_YAML = '/User/repos/demos/dask/yaml/parquet-to-dask.yaml'

HANDLER   = 'parquet_to_dask'

#### some of the following cells can be commented out if this notebook has already been run.

In [5]:
# load function from a local Python file
parq2dask = mlrun.new_function(command=FUNC_PY, 
                               image=IMAGE,
                               kind=JOB_KIND)

parq2dask.spec.remote = True
parq2dask.spec.replicas = DASK_SHARDS 
parq2dask.spec.max_replicas = DASK_SHARDS
parq2dask.spec.service_type = 'NodePort'
parq2dask.spec.image_pull_policy = 'Always'
parq2dask.spec.build.image = IMAGE

parq2dask.export(FUNC_YAML)
# parq2dask = mlrun.import_function(FUNC_YAML)

[mlrun] 2020-02-19 09:55:40,286 function spec saved to path: /User/repos/demos/dask/yaml/parquet-to-dask.yaml


In [6]:
parq2dask.apply(mlrun.mount_v3io())
parq2dask.deploy()

# create and run the task
parq_to_dask_task = mlrun.NewTask(
    TASK_NAME, 
    handler=HANDLER,  
    params={
        'parquet_url'      : DATA_PATH,
        'sample'           : DOWNSAMPLE,
        'shards'           : DASK_SHARDS,
        'threads_per'      : DASK_THREADS_PER,
        'memory_limit'     : MEMORY_LIMIT,
        'dask_key'         : DASK_KEY,
        'target_path'      : ARTIFACTS_PATH})

# run
rn = parq2dask.run(parq_to_dask_task)

[mlrun] 2020-02-19 09:55:40,305 starting remote build, image: yjbds/mlrun-dask:dev
[mlrun] 2020-02-19 09:55:40,359 starting run user-task-parq-to-dask uid=c6216f79140b44f89d6a527ccb0406e5  -> http://mlrun-api:8080
[mlrun] 2020-02-19 09:55:41,263 saving function: parquet-to-dask, tag: latest
[mlrun] 2020-02-19 09:55:46,786 dask client at: tcp://mlrun-parquet-to-dask-bac5add5-3.default-tenant:8786 dashboard: 
[mlrun] 2020-02-19 09:55:46,806 using remote dask scheduler (mlrun-parquet-to-dask-bac5add5-3) at: tcp://mlrun-parquet-to-dask-bac5add5-3.default-tenant:8786


/conda/lib/python3.6/site-packages/distributed/client.py:1071: VersionMismatchWarning: Mismatched versions found

cloudpickle
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 1.2.2   |
| scheduler | 1.1.1   |
+-----------+---------+

lz4
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 3.0.2   |
| scheduler | 2.1.6   |
+-----------+---------+

msgpack
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 0.5.6   |
| scheduler | 1.0.0   |
+-----------+---------+

numpy
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 1.17.4  |
| scheduler | 1.18.1  |
+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))
distributed.utils - ERROR - tuple is not allowed for map key
Traceback (most recent call last):
  File "/conda/lib/python3.6/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File 

[mlrun] 2020-02-19 10:00:10,490 exec error - tuple is not allowed for map key


tuple is not allowed for map key


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...0406e5,0,Feb 19 09:55:41,error,user-task-parq-to-dask,host=jupyter-byjwr2vyk0-e86jd-599b695d5b-8nn7bkind=daskowner=admin,,dask_key=airlinesmemory_limit=5GBparquet_url=/User/repos/demos/dask/dataset/partitionssample=0.01shards=8target_path=/User/repos/demos/dask/artifactsthreads_per=8,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run c6216f79140b44f89d6a527ccb0406e5 --project default , !mlrun logs c6216f79140b44f89d6a527ccb0406e5 --project default
[mlrun] 2020-02-19 10:00:10,545 run executed, status=error


RunError: tuple is not allowed for map key

### the scheduler artifact can be shared

In [ ]:
print(rn.metadata.to_yaml())

In [ ]:
rn.metadata.uid

In [ ]:
rn.outputs['scheduler']

#### What's the scheduler address?

In [ ]:
import json
json.load(open(rn.outputs['scheduler']))

### notebook cluster

Let's load the scheduler file into a cluster in this notebook using our MLRun artifact:

In [ ]:
# import dask
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

#client = Client(scheduler_file=rn.outputs['scheduler'])

#df = client.get_dataset(DASK_KEY)

### create a component 'on the fly' to summarise the table

The nice thing about having a dask cluster loaded with all your data is that you can write _quick and dirty_ jobs either in your notebook, a local file, or a github repo. Here we use a local file:

In [ ]:
# write up function in local directory
summ = mlrun.new_function(command='/User/repos/demos/dask/code/describe.py', 
                          image=IMAGE,
                          kind='job')

summ.spec.build.image = IMAGE

summ.export('/User/repos/demos/dask/yaml/describe.yaml')

summ.apply(mlrun.mount_v3io())

summ.deploy(skip_deployed=True, with_mlrun=False)

# create the task
summ_task = mlrun.NewTask(
    'user-task-my-sum', 
    handler='table_summary',  
    params={
        'dask_key'   :  DASK_KEY,
        'dask_client': 'scheduler.json',
        'target_path':  ARTIFACTS_PATH,
        'name'       : 'table-summary.csv',
        'key'        : 'table-summary'})

# run
rn2 = summ.run(summ_task)

rn2.outputs

### tests

In [ ]:
import pandas as pd
pd.read_csv(rn2.outputs['table-summary'])